<a href="https://colab.research.google.com/github/aayushbhurtel/Data_Visualization/blob/main/Quiz5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Visualization
## Aayush Bhurtel
### **Fixed over an hour count**

Here I am authenticating google bigQueries API with my account.

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Firstly, I am going to print 5 rows of data from our dataset to remind us what is in the dataset and how can we plan to perform actions according to our assignment.

In [ ]:
%%bigquery --project data-viz-big-query
SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 5

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19


#What was the second most popular starting station?

here I am using professor starting code to display number of trips and start station name side by side to compare where I can see the most popular starting station.

In [ ]:


%%bigquery --project data-viz-big-query

SELECT COUNT(*) as number_trips, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
ORDER BY number_trips DESC



,number_trips,start_station_name
0,72799,21st & Speedway @PCL
1,40635,Riverside @ S. Lamar
2,36520,City Hall / Lavaca & 2nd
3,35307,2nd & Congress
4,34758,Rainey St @ Cummings
...,...,...
188,4,Marketing Event
189,2,Eeyore's 2018
190,1,Stolen
191,1,Eeyore's 2017


Now, I will be creating a subquery to calculate the second most popular starting station. 

here I have created a query with all the data from the table in first block of code. In second block of code, I compared to second max value from the list. That is done by comparing not equals with max number which would be first max value and then again counting the total number of trips. Now, since the max is not in the list, new max will be the second most popular trip.

In [ ]:
%%bigquery --project data-viz-big-query
SELECT start_station_name,
 COUNT(*) as most_popular_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name

HAVING most_popular_trip!= (SELECT MAX(most_popular_trip) as max
FROM (
SELECT COUNT(*) as most_popular_trip, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
))
ORDER BY most_popular_trip DESC
LIMIT 1

,start_station_name,most_popular_trip
0,Riverside @ S. Lamar,40635


#Calculate the trips that lasted over an hour and were a round trip.
using professor starting code to calculate the trips that were a round trip. This code is straight forward, I just select start_station_name and duration_minutes from the database and put a condition to display only dataset where start_station_name is same as end_station_name. That way we can conclude that a biker has done a round trip.

In [ ]:


%%bigquery --project data-viz-big-query

SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name
GROUP BY start_station_name, duration_minutes
LIMIT 10




,start_station_name,duration_minutes
0,Toomey Rd @ South Lamar,31
1,Toomey Rd @ South Lamar,30
2,State Capitol @ 14th & Colorado,19
3,State Capitol @ 14th & Colorado,17
4,Waller & 6th St.,6
5,Toomey Rd @ South Lamar,0
6,Waller & 6th St.,12
7,Pease Park,47
8,State Capitol @ 14th & Colorado,0
9,State Capitol @ 14th & Colorado,6


Now I calculated the number of trips total in the dataset that was over an hour in duration. I used same code as above but added a condition that as COUNTIF where duration is over 60 minutes and ordered the data by number of trips.

In [21]:
%%bigquery --project data-viz-big-query

SELECT start_station_name, COUNT(start_station_name) as number_of_trips, COUNTIF(duration_minutes > 60) as over_an_hour_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name
GROUP BY start_station_name

,start_station_name,number_of_trips,over_an_hour_trips
0,Toomey Rd @ South Lamar,633,183
1,State Capitol @ 14th & Colorado,910,240
2,Waller & 6th St.,583,113
3,Pease Park,167,18
4,Capital Metro HQ - East 5th at Broadway,635,145
...,...,...,...
179,Republic Square @ Guadalupe & 4th St.,1090,305
180,3rd & West,2002,373
181,3rd/West,816,164
182,26th/Nueces,607,68
